## Teams  
In AutoGen, a group of two or more agents working together is referred to as a `Team`.  

AutoGen provides several pre-configured agent workflows. One of the simplest is the `RoundRobinGroupChat`.  

### RoundRobinGroupChat  
`RoundRobinGroupChat` is a straightforward workflow where all participating agents share the same context and take turns responding in a round-robin fashion.  


In [6]:
import sys
sys.path.append("..")
from model_clients.azure import get_model
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.base import TaskResult
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from rich import print


In the following cell we create the two agents that will make up the team, one is in charge of writing a poem, the other to evaluate it.

In [7]:
# Create the primary agent.
writer_agent = AssistantAgent(
    "writer",
    model_client=get_model(),
    system_message="You are a helpful AI assistant.",
)

# Create the critic agent.
critic_agent = AssistantAgent(
    "critic",
    model_client=get_model(),
    system_message="Provide constructive feedback. Respond with 'APPROVE' only when your feedbacks are addressed.",
)

We now establish communication between the two individual agents using a RoundRobinGroupChat, which facilitates the execution flow between the writer agent and the critic agent in an alternating manner. The writer agent submits content, the critic agent provides feedback, and the process continues iteratively.

To prevent an indefinite loop, we implement a text_termination rule. In this case, the rule is triggered when the word **APPROVE** appears in the agent's response, signaling the completion of the exchange.

In [8]:
# Define a termination condition that stops the task if the critic approves.
text_termination = TextMentionTermination("APPROVE")

# Create a team with the two agents and the termination condition.
team = RoundRobinGroupChat(
    [writer_agent, critic_agent], termination_condition=text_termination
)

It's now time to run the Team using the `run` method.

In [9]:
result:TaskResult = await team.run(task="Write a short poem about the fall season.")
print(result)

TaskResult(
    messages=[
        TextMessage(
            source='user',
            models_usage=None,
            metadata={},
            content='Write a short poem about the fall season.',
            type='TextMessage'
        ),
        TextMessage(
            source='writer',
            models_usage=RequestUsage(prompt_tokens=28, completion_tokens=125),
            metadata={},
            content="Golden leaves dance in the crisp, cool air,  \nWhispering secrets only autumn can share.  
\nPumpkins adorn every doorstep and lane,  \nAs harvests of plenty are gathered again.  \n\nThe sky stretches wide 
in a soft, warm hue,  \nWhile gentle winds carry sweet scents anew.  \nSweaters embrace us in comforting folds,  
\nAs stories by the fireside are lovingly told.  \n\nNature's quilt unfolds in russet and gold,  \nA tapestry of 
beauty for all to behold.  \nIn fall's fleeting moments, hearts find their pause,  \nIn the season's embrace, just 
because.  ",
            type='TextMessage'
        ),
        TextMessage(
            source='critic',
            models_usage=RequestUsage(prompt_tokens=170, completion_tokens=179),
            metadata={},
            content='Your poem beautifully captures the essence of fall, painting vivid imagery of the season with 
its colors and activities. The choice of words like "golden leaves," "crisp, cool air," and "russet and gold" 
aligns wonderfully with the theme. The structure flows seamlessly, with each stanza highlighting different aspects 
of autumn, from nature to personal experiences.\n\nConsider placing a slightly stronger emphasis on the transition 
from the first to the second stanza to ensure continuity in the imagery and feelings being portrayed. Perhaps 
introducing a word or phrase that bridges harvest to the cozy feeling would enrich the narrative.\n\nThe closing 
sentiment of "In fall\'s fleeting moments, hearts find their pause, In the season\'s embrace, just because" 
elegantly reflects the inner reflection prompted by the fall season.\n\nOnce these suggestions are addressed, you 
should have a well-rounded poem. Would you like to make any adjustments based on the feedback?',
            type='TextMessage'
        ),
        TextMessage(
            source='writer',
            models_usage=RequestUsage(prompt_tokens=342, completion_tokens=192),
            metadata={},
            content="Thank you for the thoughtful feedback. Here's a revised version that incorporates your 
suggestion:\n\nGolden leaves dance in the crisp, cool air,  \nWhispering secrets only autumn can share.  \nAs 
nature dons its vibrant attire,  \nHarvests of plenty set hearts afire.  \n\nPumpkins adorn every doorstep and 
lane,  \nApple-scented breezes call out once again.  \nSweaters embrace us in comforting folds,  \nAs stories by 
the fireside are lovingly told.  \n\nThe sky stretches wide in a soft, warm hue,  \nBridging the coziness felt in 
all we do.  \nNature's quilt unfolds in russet and gold,  \nA tapestry of beauty for all to behold.  \n\nIn fall's 
fleeting moments, hearts find their pause,  \nIn the season's embrace, just because.  \n\nI hope this version 
better bridges the ideas between stanzas while maintaining the poem's essence. Let me know if further changes are 
needed!",
            type='TextMessage'
        ),
        TextMessage(
            source='critic',
            models_usage=RequestUsage(prompt_tokens=551, completion_tokens=106),
            metadata={},
            content='The revised poem effectively bridges the initial vivid imagery of autumn\'s natural beauty 
with the cozy experiences it brings. The transition from "vibrant attire" to "harvests of plenty" provides a smooth
connection between the themes. The inclusion of "Apple-scented breezes" further enriches the sensory experience, 
adding depth to the narrative.\n\nYour adjustments enhance the poem\'s flow, and the continuity between the stanzas
is now more cohesive while still capturing the warm

We can also use AutoGen's `Console` class to generate a more structured output.  

It is important to note that a `Team`, like individual agents, maintains its state. Therefore, if the next task is unrelated to the previous one, it is generally recommended to reset the team using the `reset` method before starting a new run.  


In [5]:
await team.reset()  # Reset the team for a new task.
await Console(
    team.run_stream(task="Write a joke about programming in Python.")
)  

---------- user ----------
Write a joke about programming in Python.
---------- writer ----------
Why did the Python programmer break up with their string?

It wasn't her type!
---------- critic ----------
This is a clever joke that utilizes programming terminology effectively. It plays on the concept of "types" in programming languages and relates it to a personal relationship, which is relatable and humorous. However, you might want to consider adding a bit more context for those unfamiliar with programming terms to make it accessible to a wider audience. Here's a suggestion:

Why did the Python programmer break up with their string?

They realized she wasn't their type—literally! (In Python, strings have a specific 'type' just like integers and floats.)

Feel free to revise or expand upon this idea for broader engagement. 

Please make changes based on this feedback for approval.
---------- writer ----------
Why did the Python programmer break up with their string?

They realized sh

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='Write a joke about programming in Python.', type='TextMessage'), TextMessage(source='writer', models_usage=RequestUsage(prompt_tokens=27, completion_tokens=17), metadata={}, content="Why did the Python programmer break up with their string?\n\nIt wasn't her type!", type='TextMessage'), TextMessage(source='critic', models_usage=RequestUsage(prompt_tokens=61, completion_tokens=131), metadata={}, content='This is a clever joke that utilizes programming terminology effectively. It plays on the concept of "types" in programming languages and relates it to a personal relationship, which is relatable and humorous. However, you might want to consider adding a bit more context for those unfamiliar with programming terms to make it accessible to a wider audience. Here\'s a suggestion:\n\nWhy did the Python programmer break up with their string?\n\nThey realized she wasn\'t their type—literally! (In Python, s

Together with `TextTermination`is it possible to interrupt a team any time using [ExternalTermination](https://microsoft.github.io/autogen/stable/user-guide/agentchat-user-guide/tutorial/teams.html#stopping-a-team) 
or one of the [other](https://microsoft.github.io/autogen/stable/user-guide/agentchat-user-guide/tutorial/termination.html#termination) termination conditions.